In [ ]:
---
title: "Assignment 17"
author: "Anthony J. Kulowski"
date: "November 14, 2018"
output: word_document
---

##Prepare Dataset
```{r}
#Load packages
library(ggplot2)
library(caret)
library(lattice)
library(RANN)

#Read and import car data saved as "df""
car_data = read.csv("C:/Users/student/Documents/Bryant University/4 - Senior Year 2018-19/2018 FA/MATH 421/c2015.csv")
df = read.csv("c2015.csv", header=TRUE)

#For loop to make col names all lowercase
for(x in 1:ncol(df)) {
  colnames(df)[x] <- tolower(colnames(df)[x])
}

df$year <- NULL
df$st_case <- NULL
df$trav_sp <- NULL
df$age = as.numeric(df$age)
df$month = as.factor(df$month)
df$mod_year = as.numeric(df$mod_year)

df = df[df$hour != 99,]
df = df[df$sex != "Unknown",]
df = df[df$sex != "Not Rep",]
df = df[df$mod_year != "Unknown",]
df = df[df$mod_year != "Not Rep",]
df = df[df$deformed != "Unknown",]
df = df[df$deformed != "Not Reported",]
df = df[df$inj_sev != "Unknown",]
df = df[df$man_coll != "Unknown",]
df = df[df$owner != "Unknown",]
df = df[df$route != "Unknown",]
df = df[df$lgt_cond != "Unknown",]
df = df[df$lgt_cond != "Not Reported",]
df = df[df$weather != "Unkno",]
df = df[df$weather != "Not R",]

##df = QuickClean(df, 1)
sum(is.na(df))

```

##Change target variable level
```{r}
levels(df$inj_sev) = c("survived","died","survived","survived","survived","survived","survived","survived")
```

##Impute missing values data
```{r}
preProcess_med <- preProcess(df, method='medianImpute')
MedData <- predict(preProcess_med, newdata = df)
```

##Check for missing values
```{r}
if(sum(is.na(MedData)) == 0) {
  print('0')
} else {
    for(x in 1:ncol(MedData)) {
      miss1 <- sum(is.na(MedData[x]))
      print(colnames(MedData)[x])
      print(miss1)
    }
  }
```

##Data Visualization
```{r}
ggplot(data = MedData) + geom_bar(mapping = aes(x = inj_sev, fill = weather), position = "dodge")

ggplot(data = MedData) + geom_bar(mapping = aes(x = inj_sev, fill = sex)) + facet_wrap(~sex)

ggplot(data = MedData) + geom_density(mapping=aes(x = age, fill = inj_sev)) + facet_wrap(~inj_sev)
```


##Predictive Models
```{r}
set.seed(41)

#Decision Tree
dt1 <- train(inj_sev ~.,data = MedData, method = "rpart", 
               trControl = trainControl(method ="cv", number = 3, verboseIter =  TRUE))

#Random Forest
rf1 <- train(inj_sev ~.,data = MedData, method = "ranger", 
               trControl = trainControl(method ="cv", number = 3, verboseIter = TRUE))

#GLM
glm1 = train(inj_sev~., data = MedData, method="glmnet")

#Decision Tree Tuned
myGrid1 = expand.grid(.cp=0.02)

dt2 <- train(inj_sev ~.,data = MedData, method = "rpart", 
               trControl = trainControl(method ="cv", number = 3, verboseIter = TRUE),
               tuneGrid = myGrid1)


#Random Forest Tuned
myGrid2 = expand.grid(mtry = c(1:2), splitrule = c("gini"),
                     min.node.size = c(1:2))

rf2 <- train(inj_sev ~.,data = MedData, method = "ranger", 
               trControl = trainControl(method ="cv", number = 3, verboseIter = TRUE),
               tuneGrid = myGrid2)

#GLM Tuned
myGrid3 = expand.grid(.alpha=0:1,.lambda=seq(0.001,1,length=3))
glm2 = train(inj_Sev~., data = MedData, method="glmnet", tuneGrid=myGrid3)
```

##Decision Tree Accuracy
```{r}
dt1
dt2
```

##Random Forest Accuracy
```{r}
rf1
rf2
```

##GLM Accuracy
```{r}
glm1
glm2
```